In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
train9_df = pd.read_csv("D:/R_S_DataAnalytics/train9.csv")


In [3]:
F=train9_df.sort_values(['kda_ratio','hero_id'], ascending=[False,True])
F.head(2)

,user_id,hero_id,id,num_games,num_wins,kda_ratio
5549,880,41,880_41,1,1,38000.0
1907,304,22,304_22,1,1,32000.0


In [4]:
hero_df = pd.read_csv("D:/R_S_DataAnalytics/hero_data.csv")
hero_df.head(2)

,hero_id,primary_attr,attack_type,roles,base_health,base_health_regen,base_mana,base_mana_regen,base_armor,base_magic_resistance,...,base_agility,base_intelligence,strength_gain,agility_gain,intelligence_gain,attack_range,projectile_speed,attack_rate,move_speed,turn_rate
0,1,agi,Melee,Carry:Escape:Nuker,200,1.50,75,0.9,-1.0,25,...,22,12,1.3,2.8,1.8,150,0,1.45,310,0.5
1,2,str,Melee,Initiator:Durable:Disabler:Jungler,200,4.25,75,0.9,-1.0,25,...,20,18,2.8,2.2,1.6,150,900,1.70,290,0.6


In [6]:
h_df=pd.get_dummies(hero_df, columns=['primary_attr','attack_type'],drop_first=False)
h_df.head(2)

,hero_id,roles,base_health,base_health_regen,base_mana,base_mana_regen,base_armor,base_magic_resistance,base_attack_min,base_attack_max,...,attack_range,projectile_speed,attack_rate,move_speed,turn_rate,primary_attr_agi,primary_attr_int,primary_attr_str,attack_type_Melee,attack_type_Ranged
0,1,Carry:Escape:Nuker,200,1.50,75,0.9,-1.0,25,27,31,...,150,0,1.45,310,0.5,1,0,0,1,0
1,2,Initiator:Durable:Disabler:Jungler,200,4.25,75,0.9,-1.0,25,24,28,...,150,900,1.70,290,0.6,0,0,1,1,0


In [7]:
R=pd.DataFrame(h_df.roles.str.split(':'))
P=R[['roles']].join(R.roles.str.join('|').str.get_dummies().add_prefix('roles_'))

In [60]:
P.drop(['roles'],axis=1)
Q=pd.concat([h_df,P.drop(['roles'],axis=1)],axis=1)
Q.head()

,hero_id,roles,base_health,base_health_regen,base_mana,base_mana_regen,base_armor,base_magic_resistance,base_attack_min,base_attack_max,...,attack_type_Ranged,roles_Carry,roles_Disabler,roles_Durable,roles_Escape,roles_Initiator,roles_Jungler,roles_Nuker,roles_Pusher,roles_Support
0,1,Carry:Escape:Nuker,200,1.50,75,0.9,-1.0,25,27,31,...,0,1,0,0,1,0,0,1,0,0
1,2,Initiator:Durable:Disabler:Jungler,200,4.25,75,0.9,-1.0,25,24,28,...,0,0,1,1,0,1,1,0,0,0
2,3,Support:Disabler:Nuker:Durable,200,1.50,75,0.9,1.0,25,37,43,...,1,0,1,1,0,0,0,1,0,1
3,4,Carry:Disabler:Jungler:Nuker:Initiator,200,1.50,75,0.9,0.0,25,33,39,...,0,1,1,0,0,1,1,1,0,0
4,5,Support:Disabler:Nuker:Jungler,200,1.50,75,0.9,-1.0,25,19,25,...,1,0,1,0,0,0,1,1,0,1


In [11]:
A=train9_df.merge(Q, how='outer',on='hero_id')
A.head(10)

,user_id,hero_id,id,num_games,num_wins,kda_ratio,roles,base_health,base_health_regen,base_mana,...,attack_type_Ranged,roles_Carry,roles_Disabler,roles_Durable,roles_Escape,roles_Initiator,roles_Jungler,roles_Nuker,roles_Pusher,roles_Support
0,1,74,1_74,304,185,3743.699732,Carry:Nuker:Disabler:Escape:Pusher,200,1.5,75,...,1,1,1,0,1,0,0,1,1,0
1,2,74,2_74,657,362,3061.059634,Carry:Nuker:Disabler:Escape:Pusher,200,1.5,75,...,1,1,1,0,1,0,0,1,1,0
2,5,74,5_74,276,160,3899.408284,Carry:Nuker:Disabler:Escape:Pusher,200,1.5,75,...,1,1,1,0,1,0,0,1,1,0
3,6,74,6_74,295,166,3802.301963,Carry:Nuker:Disabler:Escape:Pusher,200,1.5,75,...,1,1,1,0,1,0,0,1,1,0
4,8,74,8_74,780,459,3830.321521,Carry:Nuker:Disabler:Escape:Pusher,200,1.5,75,...,1,1,1,0,1,0,0,1,1,0
5,12,74,12_74,620,368,4649.333333,Carry:Nuker:Disabler:Escape:Pusher,200,1.5,75,...,1,1,1,0,1,0,0,1,1,0
6,17,74,17_74,256,111,3618.459302,Carry:Nuker:Disabler:Escape:Pusher,200,1.5,75,...,1,1,1,0,1,0,0,1,1,0
7,22,74,22_74,614,331,3710.166469,Carry:Nuker:Disabler:Escape:Pusher,200,1.5,75,...,1,1,1,0,1,0,0,1,1,0
8,24,74,24_74,261,122,3772.413793,Carry:Nuker:Disabler:Escape:Pusher,200,1.5,75,...,1,1,1,0,1,0,0,1,1,0
9,27,74,27_74,190,103,3392.586352,Carry:Nuker:Disabler:Escape:Pusher,200,1.5,75,...,1,1,1,0,1,0,0,1,1,0


In [29]:
new_feat_test=A.groupby("user_id").agg("mean").reset_index()

In [30]:
# i have only selected the average of the numeric data but not the categorical data
new_feat_test1=new_feat_test[['user_id',
       'base_health_regen', 'base_armor',
       'base_attack_min', 'base_attack_max',
       'base_strength', 'base_agility', 'base_intelligence', 'strength_gain',
       'agility_gain', 'intelligence_gain', 'attack_range', 'projectile_speed',
       'attack_rate', 'move_speed', 'turn_rate','kda_ratio']]

In [32]:
new_feat_test1.head(2)

,user_id,base_health_regen,base_armor,base_attack_min,base_attack_max,base_strength,base_agility,base_intelligence,strength_gain,agility_gain,intelligence_gain,attack_range,projectile_speed,attack_rate,move_speed,turn_rate,kda_ratio
0,1,2.055556,-0.333333,23.888889,30.555556,20.888889,17.555556,19.555556,2.522222,1.822222,2.311111,305.000000,988.888889,1.711111,294.444444,0.633333,3857.506249
1,2,1.638889,0.166667,21.666667,28.111111,19.222222,20.000000,20.111111,2.355556,2.022222,2.244444,366.666667,800.000000,1.655556,301.111111,0.588889,2857.903926


In [33]:
# applying groupby on hero_id and try to get the average of "kda_ratio" per hero
 
k_h=A.groupby("hero_id")["kda_ratio"].agg("mean")
k_he=pd.DataFrame(k_h)
k_he=k_he.rename(columns={"kda_ratio":"k_hero"})
k_he.reset_index(inplace=True)

# applying groupby on hero_id and try to get the average of "per_wins" per hero

# p_w=A.groupby("hero_id")["per_wins"].agg("mean")
# p_w_h=pd.DataFrame(p_w)
# p_w_h=p_w_h.rename(columns={"per_wins":"per_wins_hero"})
# p_w_h.reset_index(inplace=True)

In [39]:
train1_df.head(2)

,user_id,hero_id,id,num_games,num_wins,kda_ratio
0,1,41,1_41,201,125,3731.707317
1,2,17,2_17,427,234,2922.058381


In [42]:
# merging of the above found properties per hero 
train1_df=pd.merge(train1_df,k_he,how="left",on="hero_id")
# df1_train=pd.merge(df1_train,p_w_h,how="left",on="hero_id")
train1_df=pd.merge(train1_df,new_feat_test1,how="left",on="user_id")

train1_df.head(2)

,user_id,hero_id,id,num_games,num_wins,kda_ratio_x,k_hero_x,base_health_regen_x,base_armor_x,base_attack_min_x,...,base_intelligence_y,strength_gain_y,agility_gain_y,intelligence_gain_y,attack_range_y,projectile_speed_y,attack_rate_y,move_speed_y,turn_rate_y,kda_ratio
0,1,41,1_41,201,125,3731.707317,3889.420457,2.055556,-0.333333,23.888889,...,19.555556,2.522222,1.822222,2.311111,305.000000,988.888889,1.711111,294.444444,0.633333,3857.506249
1,2,17,2_17,427,234,2922.058381,3964.276777,1.638889,0.166667,21.666667,...,20.111111,2.355556,2.022222,2.244444,366.666667,800.000000,1.655556,301.111111,0.588889,2857.903926


In [46]:
# another merge on herod and df1_train
kk=pd.merge(train1_df,Q[['hero_id', 'base_health', 'base_health_regen', 'base_mana',
       'base_mana_regen', 'base_armor', 'base_magic_resistance',
       'base_attack_min', 'base_attack_max', 'base_strength', 'base_agility',
       'base_intelligence', 'strength_gain', 'agility_gain',
       'intelligence_gain', 'attack_range', 'projectile_speed', 'attack_rate',
       'move_speed', 'turn_rate', 'primary_attr_int', 'primary_attr_str',
       'attack_type_Ranged', 'roles_Disabler', 'roles_Durable', 'roles_Escape',
       'roles_Initiator', 'roles_Jungler', 'roles_Nuker', 'roles_Pusher',
       'roles_Support', 'roles_Carry']],how="left",on="hero_id")

In [48]:
kk.head()

,user_id,hero_id,id,num_games,num_wins,kda_ratio_x,k_hero_x,base_health_regen_x,base_armor_x,base_attack_min_x,...,attack_type_Ranged,roles_Disabler,roles_Durable,roles_Escape,roles_Initiator,roles_Jungler,roles_Nuker,roles_Pusher,roles_Support,roles_Carry
0,1,41,1_41,201,125,3731.707317,3889.420457,2.055556,-0.333333,23.888889,...,0,1,1,1,1,0,0,0,0,1
1,2,17,2_17,427,234,2922.058381,3964.276777,1.638889,0.166667,21.666667,...,1,1,0,1,1,0,1,0,0,1
2,5,106,5_106,240,130,5116.682739,4678.101919,1.777778,0.333333,22.555556,...,0,1,0,1,1,0,1,0,0,1
3,6,65,6_65,206,122,4131.553861,3747.512801,1.638889,-0.222222,24.777778,...,1,1,0,1,1,1,0,0,0,0
4,7,69,7_69,697,382,2474.469306,3091.711347,1.611111,-0.111111,26.555556,...,0,1,1,0,1,0,1,0,0,1


In [49]:
kk_target=kk.kda_ratio

In [53]:
# to prepare for the data for standard scaling
kk_x=kk[['k_hero_x',
        'base_health_regen_x', 'base_armor_x',
       'base_attack_min_x', 'base_attack_max_x', 'base_strength_x',
       'base_agility_x', 'base_intelligence_x', 'strength_gain_x',
       'agility_gain_x', 'intelligence_gain_x', 'attack_range_x',
       'projectile_speed_x', 'attack_rate_x', 'move_speed_x', 'turn_rate_x',
        'kda_ratio_x', 'base_health', 'base_health_regen_y',
       'base_mana', 'base_mana_regen', 'base_armor_y', 'base_magic_resistance',
       'base_attack_min_y', 'base_attack_max_y', 'base_strength_y',
       'base_agility_y', 'base_intelligence_y', 'strength_gain_y',
       'agility_gain_y', 'intelligence_gain_y', 'attack_range_y',
       'projectile_speed_y', 'attack_rate_y', 'move_speed_y', 'turn_rate_y']]


In [54]:
# Standard scaling is apllied
from sklearn.preprocessing import StandardScaler
mm=StandardScaler()
mm.fit(kk_x)
kk_scaled=mm.transform(kk_x)

In [55]:
kk_scaled=pd.DataFrame(kk_scaled,columns=['k_hero_x',
        'base_health_regen_x', 'base_armor_x',
       'base_attack_min_x', 'base_attack_max_x', 'base_strength_x',
       'base_agility_x', 'base_intelligence_x', 'strength_gain_x',
       'agility_gain_x', 'intelligence_gain_x', 'attack_range_x',
       'projectile_speed_x', 'attack_rate_x', 'move_speed_x', 'turn_rate_x',
        'kda_ratio_x', 'base_health', 'base_health_regen_y',
       'base_mana', 'base_mana_regen', 'base_armor_y', 'base_magic_resistance',
       'base_attack_min_y', 'base_attack_max_y', 'base_strength_y',
       'base_agility_y', 'base_intelligence_y', 'strength_gain_y',
       'agility_gain_y', 'intelligence_gain_y', 'attack_range_y',
       'projectile_speed_y', 'attack_rate_y', 'move_speed_y', 'turn_rate_y'])
kk_scaled.head()

,k_hero_x,base_health_regen_x,base_armor_x,base_attack_min_x,base_attack_max_x,base_strength_x,base_agility_x,base_intelligence_x,strength_gain_x,agility_gain_x,...,base_agility_y,base_intelligence_y,strength_gain_y,agility_gain_y,intelligence_gain_y,attack_range_y,projectile_speed_y,attack_rate_y,move_speed_y,turn_rate_y
0,0.537692,1.957741,-0.338282,-0.093168,-0.513810,1.043036,-0.518005,0.267523,0.763750,-0.819966,...,-0.518005,0.267523,0.763750,-0.819966,0.630454,-0.515135,1.044071,1.123470,-0.627116,0.376773
1,0.661895,-0.332488,0.616670,-0.970784,-1.481678,-0.246239,0.931692,0.589548,-0.062430,0.053941,...,0.931692,0.589548,-0.062430,0.053941,0.364206,0.276281,-0.258178,-0.588095,1.194006,-0.603153
2,1.846285,0.430922,0.934987,-0.619738,-1.085732,-1.019803,1.854227,-1.278195,-1.769871,1.995957,...,1.854227,-1.278195,-1.769871,1.995957,-0.345790,-0.622083,-0.947604,-2.470816,-0.171835,0.376773
3,0.302237,-0.332488,-0.126070,0.257878,0.806011,0.097568,-1.242854,0.589548,-0.200127,-0.674315,...,-1.242854,0.589548,-0.200127,-0.674315,0.918890,1.816333,0.833413,0.438844,-0.323596,-1.338097
4,-0.785880,-0.485170,0.086141,0.959971,1.421927,-0.160287,0.206843,0.975977,-0.282745,-0.334462,...,0.206843,0.975977,-0.282745,-0.334462,0.430768,-0.144382,1.044071,1.294626,0.283445,0.621755


In [12]:
from sklearn.svm import SVR
import numpy as np
svr_rbf = SVR(kernel='rbf', C=100000, gamma=0.1)

svr_rbf.fit(kk_scaled,kk_target)
pred=svr_rbf.predict(kk_scaled)
np.sqrt(np.sum(pred-kk_target)**2)

NameError: name 'kk_scaled' is not defined

In [59]:
test9_df = pd.read_csv("D:/R_S_DataAnalytics/test9.csv")
test9_df.head(2)

,user_id,hero_id,id,num_games,num_wins,kda_ratio
0,3,65,3_65,330,183,3418.470419
1,3,104,3_104,231,107,2522.639069


In [63]:
Q.head()

,hero_id,roles,base_health,base_health_regen,base_mana,base_mana_regen,base_armor,base_magic_resistance,base_attack_min,base_attack_max,...,attack_type_Ranged,roles_Carry,roles_Disabler,roles_Durable,roles_Escape,roles_Initiator,roles_Jungler,roles_Nuker,roles_Pusher,roles_Support
0,1,Carry:Escape:Nuker,200,1.50,75,0.9,-1.0,25,27,31,...,0,1,0,0,1,0,0,1,0,0
1,2,Initiator:Durable:Disabler:Jungler,200,4.25,75,0.9,-1.0,25,24,28,...,0,0,1,1,0,1,1,0,0,0
2,3,Support:Disabler:Nuker:Durable,200,1.50,75,0.9,1.0,25,37,43,...,1,0,1,1,0,0,0,1,0,1
3,4,Carry:Disabler:Jungler:Nuker:Initiator,200,1.50,75,0.9,0.0,25,33,39,...,0,1,1,0,0,1,1,1,0,0
4,5,Support:Disabler:Nuker:Jungler,200,1.50,75,0.9,-1.0,25,19,25,...,1,0,1,0,0,0,1,1,0,1


In [65]:
# merging of the dataframe
df10_test=pd.merge(test9_df,Q,how="left",on="hero_id")
df10_test.head()

,user_id,hero_id,id,num_games,num_wins,kda_ratio,roles,base_health,base_health_regen,base_mana,...,attack_type_Ranged,roles_Carry,roles_Disabler,roles_Durable,roles_Escape,roles_Initiator,roles_Jungler,roles_Nuker,roles_Pusher,roles_Support
0,3,65,3_65,330,183,3418.470419,Initiator:Jungler:Disabler:Escape,200,3.25,75,...,1,0,1,0,1,1,1,0,0,0
1,3,104,3_104,231,107,2522.639069,Carry:Disabler:Initiator:Durable:Nuker,200,1.50,75,...,0,1,1,1,0,1,0,1,0,0
2,3,7,3_7,200,103,2793.255132,Support:Initiator:Disabler:Nuker,200,2.00,75,...,0,0,1,0,0,1,0,1,0,1
3,3,19,3_19,189,91,3493.495230,Carry:Nuker:Pusher:Initiator:Durable:Disabler,200,2.50,75,...,0,1,1,1,0,1,0,1,1,0
4,3,4,3_4,182,102,3586.402266,Carry:Disabler:Jungler:Nuker:Initiator,200,1.50,75,...,0,1,1,0,0,1,1,1,0,0


In [66]:
new_feat_test=df10_test.groupby("user_id").agg("mean").reset_index()

In [67]:
new_feat_test1=new_feat_test[['user_id',
       'base_health_regen', 'base_armor',
       'base_attack_min', 'base_attack_max',
       'base_strength', 'base_agility', 'base_intelligence', 'strength_gain',
       'agility_gain', 'intelligence_gain', 'attack_range', 'projectile_speed',
       'attack_rate', 'move_speed', 'turn_rate','kda_ratio']]

In [68]:
k_h=df10_test.groupby("hero_id")["kda_ratio"].agg("mean")
k_he=pd.DataFrame(k_h)
k_he=k_he.rename(columns={"kda_ratio":"k_hero"})
k_he.reset_index(inplace=True)

In [70]:
df1_test=pd.merge(test1_df,k_he,how="left",on="hero_id")
# df1_test=pd.merge(dtest,p_w_h,how="left",on="hero_id")
df1_test=pd.merge(test1_df,new_feat_test1,how="left",on="user_id")
df1_test.head(2)

,user_id,hero_id,id,num_games,base_health_regen,base_armor,base_attack_min,base_attack_max,base_strength,base_agility,base_intelligence,strength_gain,agility_gain,intelligence_gain,attack_range,projectile_speed,attack_rate,move_speed,turn_rate,kda_ratio
0,3,53,3_53,278,1.861111,0.222222,28.111111,35.222222,23.555556,14.777778,18.555556,2.900000,1.611111,2.022222,211.666667,822.222222,1.733333,294.444444,0.644444,3096.586034
1,4,17,4_17,180,1.527778,-1.000000,21.111111,29.333333,19.222222,17.000000,19.777778,2.344444,1.700000,2.388889,383.333333,808.333333,1.677778,297.777778,0.661111,3156.761161


In [58]:
test1_df = pd.read_csv("D:/R_S_DataAnalytics/test1.csv")
test1_df.head(2)

,user_id,hero_id,id,num_games
0,3,53,3_53,278
1,4,17,4_17,180


In [72]:
kk_test=pd.merge(df1_test,Q[['hero_id', 'base_health', 'base_health_regen', 'base_mana',
       'base_mana_regen', 'base_armor', 'base_magic_resistance',
       'base_attack_min', 'base_attack_max', 'base_strength', 'base_agility',
       'base_intelligence', 'strength_gain', 'agility_gain',
       'intelligence_gain', 'attack_range', 'projectile_speed', 'attack_rate',
       'move_speed', 'turn_rate', 'primary_attr_int', 'primary_attr_str',
       'attack_type_Ranged', 'roles_Disabler', 'roles_Durable', 'roles_Escape',
       'roles_Initiator', 'roles_Jungler', 'roles_Nuker', 'roles_Pusher',
       'roles_Support', 'roles_Carry']],how="left",on="hero_id")

In [77]:
kk_test.columns

Index(['user_id', 'hero_id', 'id', 'num_games', 'base_health_regen_x',
       'base_armor_x', 'base_attack_min_x', 'base_attack_max_x',
       'base_strength_x', 'base_agility_x', 'base_intelligence_x',
       'strength_gain_x', 'agility_gain_x', 'intelligence_gain_x',
       'attack_range_x', 'projectile_speed_x', 'attack_rate_x', 'move_speed_x',
       'turn_rate_x', 'kda_ratio', 'base_health', 'base_health_regen_y',
       'base_mana', 'base_mana_regen', 'base_armor_y', 'base_magic_resistance',
       'base_attack_min_y', 'base_attack_max_y', 'base_strength_y',
       'base_agility_y', 'base_intelligence_y', 'strength_gain_y',
       'agility_gain_y', 'intelligence_gain_y', 'attack_range_y',
       'projectile_speed_y', 'attack_rate_y', 'move_speed_y', 'turn_rate_y',
       'primary_attr_int', 'primary_attr_str', 'attack_type_Ranged',
       'roles_Disabler', 'roles_Durable', 'roles_Escape', 'roles_Initiator',
       'roles_Jungler', 'roles_Nuker', 'roles_Pusher', 'roles_Support',
 

In [78]:
kk_x=kk_test[['kda_ratio',
        'base_health_regen_x', 'base_armor_x',
       'base_attack_min_x', 'base_attack_max_x', 'base_strength_x',
       'base_agility_x', 'base_intelligence_x', 'strength_gain_x',
       'agility_gain_x', 'intelligence_gain_x', 'attack_range_x',
       'projectile_speed_x', 'attack_rate_x', 'move_speed_x', 'turn_rate_x',
        'kda_ratio', 'base_health', 'base_health_regen_y',
       'base_mana', 'base_mana_regen', 'base_armor_y', 'base_magic_resistance',
       'base_attack_min_y', 'base_attack_max_y', 'base_strength_y',
       'base_agility_y', 'base_intelligence_y', 'strength_gain_y',
       'agility_gain_y', 'intelligence_gain_y', 'attack_range_y',
       'projectile_speed_y', 'attack_rate_y', 'move_speed_y', 'turn_rate_y']]

In [79]:
# standard scaling of the test data and we will only transform this test data
kk_scaled_test=mm.transform(kk_x)

In [81]:
kk_scaled_test=pd.DataFrame(kk_scaled_test,columns=['kda_ratio',
        'base_health_regen_x', 'base_armor_x',
       'base_attack_min_x', 'base_attack_max_x', 'base_strength_x',
       'base_agility_x', 'base_intelligence_x', 'strength_gain_x',
       'agility_gain_x', 'intelligence_gain_x', 'attack_range_x',
       'projectile_speed_x', 'attack_rate_x', 'move_speed_x', 'turn_rate_x',
        'kda_ratio_user', 'base_health', 'base_health_regen_y',
       'base_mana', 'base_mana_regen', 'base_armor_y', 'base_magic_resistance',
       'base_attack_min_y', 'base_attack_max_y', 'base_strength_y',
       'base_agility_y', 'base_intelligence_y', 'strength_gain_y',
       'agility_gain_y', 'intelligence_gain_y', 'attack_range_y',
       'projectile_speed_y', 'attack_rate_y', 'move_speed_y', 'turn_rate_y'])
kk_scaled_test.head()

,kda_ratio,base_health_regen_x,base_armor_x,base_attack_min_x,base_attack_max_x,base_strength_x,base_agility_x,base_intelligence_x,strength_gain_x,agility_gain_x,...,base_agility_y,base_intelligence_y,strength_gain_y,agility_gain_y,intelligence_gain_y,attack_range_y,projectile_speed_y,attack_rate_y,move_speed_y,turn_rate_y
0,-0.777792,0.888967,0.722776,1.574302,1.333939,3.105875,-2.165389,-0.312121,2.636427,-1.742424,...,-0.254424,3.423364,-1.329241,-0.480113,2.982317,3.270827,1.982456,0.781157,-1.841197,-0.358171
1,-0.677948,-0.943215,-1.611551,-1.190188,-0.997744,-0.246239,-0.847482,0.396333,-0.117509,-1.354021,...,2.117808,2.843720,-2.816367,-2.227928,3.381690,1.730774,1.810100,0.781157,-3.207039,4.051496
2,-0.900115,-0.790533,0.510564,0.038474,0.322076,-0.332190,-0.847482,1.233597,-0.668296,-1.305470,...,-3.219714,-1.213790,4.123553,1.267701,-3.407650,-2.504369,0.431248,0.781157,4.988010,-0.358171
3,0.743269,-0.637852,-0.126070,-0.707499,-1.085732,-0.847900,1.524750,-0.054501,-1.549556,1.073500,...,-0.254424,-2.373078,3.627844,-0.480113,-1.610471,-2.504369,0.431248,5.402381,4.988010,8.461163
4,1.990568,0.888967,-0.762705,0.608925,0.630034,0.699229,2.183703,-1.149385,-1.164005,0.927848,...,-2.626656,-1.793434,-0.833533,-0.480113,7.375419,3.270827,0.431248,0.781157,-4.572880,-2.563005


In [104]:
df_f=kk_test.drop(['kda_ratio'],axis=1)
df_f.head()

,user_id,hero_id,id,num_games,base_health_regen_x,base_armor_x,base_attack_min_x,base_attack_max_x,base_strength_x,base_agility_x,...,attack_type_Ranged,roles_Disabler,roles_Durable,roles_Escape,roles_Initiator,roles_Jungler,roles_Nuker,roles_Pusher,roles_Support,roles_Carry
0,3,53,3_53,278,1.861111,0.222222,28.111111,35.222222,23.555556,14.777778,...,1,0,0,1,0,1,1,1,0,1
1,4,17,4_17,180,1.527778,-1.000000,21.111111,29.333333,19.222222,17.000000,...,1,1,0,1,1,0,1,0,0,1
2,9,51,9_51,70,1.555556,0.111111,24.222222,32.666667,19.111111,17.000000,...,0,1,1,0,1,0,1,0,0,0
3,10,54,10_54,189,1.583333,-0.222222,22.333333,29.111111,18.444444,21.000000,...,0,1,1,1,0,1,0,0,0,1
4,15,74,15_74,263,1.861111,-0.555556,25.666667,33.444444,20.444444,22.111111,...,1,1,0,1,0,0,1,1,0,1


In [101]:
Z=kk_test['kda_ratio']
Z.head()

0    3096.586034
1    3156.761161
2    3022.862616
3    4013.320345
4    4765.059940
Name: kda_ratio, dtype: float64

In [84]:
kk_scaled_test=pd.concat([kk_scaled_test,kk_test[['hero_id','primary_attr_int', 'primary_attr_str', 'attack_type_Ranged',
       'roles_Disabler', 'roles_Durable', 'roles_Escape', 'roles_Initiator',
       'roles_Jungler', 'roles_Nuker', 'roles_Pusher', 'roles_Support',
       'roles_Carry']]],axis=1)

In [88]:
# predicting the result


kk_scaled_test=mm.transform(kk_x)
pred=svr_rbf.predict(kk_scaled_test)


In [92]:
pd.concat([pd.DataFrame(svr_rbf.predict(kk_scaled_test)).rename(columns={0:"kda_ratio"}),pd.DataFrame(kk_test.id)],axis=1).to_csv("D://R_S_DataAnalytics//A_V_submission.csv")

In [94]:
pd.concat([pd.DataFrame(svr_rbf.predict(kk_scaled_test)).rename(columns={0:"kda_ratio"}),pd.DataFrame(kk_test.id)],axis=1).head()

,kda_ratio,id
0,3465.722488,3_53
1,3465.937616,4_17
2,3466.738875,9_51
3,3467.150439,10_54
4,3467.158788,15_74


In [107]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor(random_state=42)
forest_reg.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [108]:
print('Random Forest R squared": %.4f' % forest_reg.score(X_test, y_test))

Random Forest R squared": 0.1005


In [110]:
import numpy as np
from sklearn.metrics import mean_squared_error
lin_mse = mean_squared_error(y_pred, y_test)
lin_rmse = np.sqrt(lin_mse)
print('Linear Regression RMSE: %.4f' % lin_rmse)

Linear Regression RMSE: 585.7079


In [111]:
y_pred = forest_reg.predict(X_test)
forest_mse = mean_squared_error(y_pred, y_test)
forest_rmse = np.sqrt(forest_mse)
print('Random Forest RMSE: %.4f' % forest_rmse)

Random Forest RMSE: 585.7079


In [105]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_f, Z, test_size=0.4, random_state=0)
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [106]:
y_pred = regressor.predict(X_test)
print('Linear Regression R squared": %.4f' % regressor.score(X_test, y_test))

Linear Regression R squared": 0.1935


In [112]:
from sklearn import ensemble
from sklearn.ensemble import GradientBoostingRegressor
model = ensemble.GradientBoostingRegressor()
model.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [113]:
print('Gradient Boosting R squared": %.4f' % model.score(X_test, y_test))

Gradient Boosting R squared": 0.1277


In [114]:
y_pred = model.predict(X_test)
model_mse = mean_squared_error(y_pred, y_test)
model_rmse = np.sqrt(model_mse)
print('Gradient Boosting RMSE: %.4f' % model_rmse)

Gradient Boosting RMSE: 576.7881


In [11]:
# # Use PolynomialFeatures in sklearn.preprocessing to create two-way interactions for all features
# from itertools import combinations
# from sklearn.preprocessing import PolynomialFeatures

# def add_interactions(F):
#     # Get feature names
#     combos = list(combinations(list(F.columns), 2))
#     colnames = list(F.columns) + ['_'.join(x) for x in combos]
    
#     # Find interactions
#     poly = PolynomialFeatures(interaction_only=True, include_bias=False)
#     F = poly.fit_transform(F)
#     F = pd.DataFrame(F)
#     F.columns = colnames
    
#     # Remove interaction terms with all 0 values            
#     noint_indicies = [i for i, x in enumerate(list((F == 0).all())) if x]
#     F = F.drop(F.columns[noint_indicies], axis=1)
    
#     return F

In [10]:
# F = add_interactions(F)
# print(F.head(5))

In [7]:
# # Use PCA from sklearn.decompostion to find principal components
# from sklearn.decomposition import PCA

# pca = PCA(n_components=10)
# F_pca = pd.DataFrame(pca.fit_transform(F))

In [8]:
# print(F_pca.head(10))

In [9]:
# y=F.kda_ratio
# ly

In [30]:
# K=F.drop(['kda_ratio'], axis=1).head()

In [1]:
# #now we will use train test split and model 
# from sklearn.cross_validation import train_test_split
# X_train,X_test,Y_tain,Y_test=train_test_split(K,y, train_size=0.70, random_state=1)

In [2]:
# corr_matrix = train9_df.corr()
# corr_matrix["kda_ratio"].sort_values(ascending=False)

In [3]:
# F.info()

In [4]:
# sns.distplot(train9_df['kda_ratio']);
# plt.show()
# res = stats.probplot(train9_df['kda_ratio'], plot=plt)

In [15]:
# train9_df['kda_ratio'] = np.log(train9_df['kda_ratio'])

In [16]:
# sns.distplot(train9_df['kda_ratio']);
# plt.show()
# res = stats.probplot(train9_df['kda_ratio'], plot=plt)

In [5]:
# # train9_df.isnull().sum().sort_values(ascending=False)
# # percent = (train9_df.isnull().sum()/train9_df.isnull().count()).sort_values(ascending=False)
# # percent
# #correlation matrix
# corrmat = train9_df.corr()
# f, ax = plt.subplots(figsize=(7, 8))
# sns.heatmap(corrmat, vmax=.8, square=True);
# plt.show()

In [86]:
# H=G['kda_ratio']


In [18]:
# corr_matrix = train1_df.corr()
# corr_matrix["kda_ratio"].sort_values(ascending=False)

In [19]:
# sns.distplot(train1_df['kda_ratio']);
# plt.show()
# res = stats.probplot(train1_df['kda_ratio'], plot=plt)

In [20]:
# #correlation matrix
# corrmat = train1_df.corr()
# f, ax = plt.subplots(figsize=(7, 8))
# sns.heatmap(corrmat, vmax=.8, square=True);
# plt.show()

In [34]:
# train1_df.sort_values(['num_wins'], axis=1 , ascending=False)
# sorted(list, key=lambda trian1_df: (train_df['id'], train_df['kda.ratio']))

In [10]:
# df1 = pd.merge(train1_df,df, how='inner',left_on='None', right_on='None')

In [21]:
# train1_df.isnull().sum().sort_values(ascending=False)
# percent = (train1_df.isnull().sum()/train1_df.isnull().count()).sort_values(ascending=False)
# percent

In [21]:
# sns.distplot(test9_df['kda_ratio']);
# plt.show()
# res = stats.probplot(test9_df['kda_ratio'], plot=plt)

In [22]:
# # test9_df.isnull().sum().sort_values(ascending=False)
# # percent = (test9_df.isnull().sum()/test9_df.isnull().count()).sort_values(ascending=False)
# # percent
# #correlation matrix
# corrmat = test9_df.corr()
# f, ax = plt.subplots(figsize=(7, 8))
# sns.heatmap(corrmat, vmax=.8, square=True);
# plt.show()

In [90]:
# from sklearn.model_selection import KFold
# from sklearn.model_selection import train_test_split

In [91]:
# X_train9_df=train9_df[:-50]
# X_train1_df=train1_df[:-50]
# X_test9_df=train9_df[-50:]
# X_test1_df=train1_df[-50:]

# Y_train9_df = train9_df.kda_ratio[:-50]
# Y_train1_df = train1_df.kda_ratio[:-50]
# Y_test9_df = train9_df.kda_ratio[-50:]
# Y_test9_df = train1_df.kda_ratio[-50:]

# print (X_train9_df.shape)
# print (X_train1_df.shape)
# print (X_test9_df.shape)
# print (X_test1_df.shape)

# print (Y_train9_df.shape)
# print (Y_train1_df.shape)
# print (Y_test9_df.shape)
# print (Y_test9_df.shape)

In [115]:
# # test1_df.isnull().sum().sort_values(ascending=False)
# # percent = (test1_df.isnull().sum()/test1_df.isnull().count()).sort_values(ascending=False)
# # percent
# #correlation matrix
# corrmat = test1_df.corr()
# f, ax = plt.subplots(figsize=(7, 8))
# sns.heatmap(corrmat, vmax=.8, square=True);
# plt.show()

In [116]:
# #correlation matrix
# corrmat = hero_df.corr()
# f, ax = plt.subplots(figsize=(7, 8))
# sns.heatmap(corrmat, vmax=.8, square=True);
# plt.show()

In [18]:
# hero_df.isnull().sum().sort_values(ascending=False)
# percent = (hero_df.isnull().sum()/hero_df.isnull().count()).sort_values(ascending=False)
# percent